<a href="https://colab.research.google.com/github/uob-positron-imaging-centre/PEPT-Algorithms-RoPP/blob/main/PEPTEM_RoPP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a target="_blank"  href="https://github.com/uob-positron-imaging-centre/pept"><img src="https://github.com/uob-positron-imaging-centre/misc-hosting/blob/master/logo.png?raw=true" style="height:200px; display: block; margin-left: auto; margin-right: auto;"/></a>

# Interactive PEPT Analysis Examples using *PEPT-EM*

> [1] Manger S, Renaud A, Vanneste J. An Expectation-Maximization Algorithm for Positron Emission Particle Tracking https://arxiv.org/abs/2104.07457

---

#### Copyright 2021 the `pept` developers
##### Jupyter Notebook authored by Sam Manger, Andrei Leonard Nicusan and Dr. Kit Windows-Yule for the "PEPT: A Comparative Review" paper, commissioned by the Reports on Progress in Physics journal

Licensed under the GNU License, Version 3.0 (the "License").

---


# 1. Introduction

Positron emission particle tracking (PEPT) is a powerful technique allowing the non-invasive, three-dimensional tracking of one or more radioactive 'tracer' particles through particulate, fluid or multiphase systems. It allows particle or fluid motion to be tracked with sub-millimetre accuracy and sub-millisecond temporal resolution and, due to its use of highly-penetrating 511keV gamma rays, can be used to probe the internal dynamics of even large, dense, optically opaque systems <sup>[[2]](https://www.sciencedirect.com/science/article/pii/016890029390864E) [[3]](https://www.sciencedirect.com/science/article/pii/S0263876208003341) [[4]](https://aip.scitation.org/doi/abs/10.1063/1.4983046@rsi.2017.IMGP2017.issue-1)</sup>. In light of its versatility both in terms of the scales and materials of particles which can be tracked <sup>[[5]](https://www.sciencedirect.com/science/article/pii/S1672251507001455)[[6]](https://www.sciencedirect.com/science/article/pii/S0168900206005341)</sup>, and the sizes and geometries of the systems which can be imaged <sup>[[7]](https://www.sciencedirect.com/science/article/pii/S0168900209001880) [[8]](https://www.sciencedirect.com/science/article/pii/S0029549316000273)</sup> , the technique has wide-ranging applicability in diverse scientific, industrial and biomedical applications.

PEPT is performed by radioactively labelling a particle with a positron-emitting radioisotope such as Fluorine-18 ($^{18}\mathrm{F}$) or Gallium-68 ($^{68}\mathrm{Ga}$), and using the back-to-back gamma rays produced by electron-positron annihilation events in and around the tracer to triangulate its spatial position. Each detected gamma ray represents a **line of response (LoR)** .

## 1.1. This Jupyter Notebook

This interactive Jupyter Notebook illustrates the main processing steps employed by PEPT-EM<sup>[1]</sup> for radioactive tracer tracking, as described in the Reports on Progress in Physics "PEPT: A Comparative Review" paper.

An [example dataset](https://raw.githubusercontent.com/uob-positron-imaging-centre/example_data/master/sample_1p_fluidised_bed.csv) is used from an experiment run at the University of Birmingham Positron Imaging Centre using the ADAC Forté by Matthew Herald. It consists of a single 1 mm diameter MCC particle activated with Fluorine-18 radioactive tracer material inside a bubbling fluidised bed. The fluidised bed was filled with 90% sand and 10% MCC; air was fed into the bottom of the bed at a rate of 37 litres per minute at 3.5 bar. This dataset was chosen for its high quality captured lines of response, with the tracer still depicting the random particle motion that is inherent to bubbling fluidised beds - and typical in Lagrangian particle tracking.

The [`pept`](https://github.com/uob-positron-imaging-centre/pept) Python library is used for initialising and visualising PEPT data. While not required *per se* for illustrating PEPT algorithms' processing steps, it significantly reduces the amount of repetitive code and visual noise, allowing the reader to focus on the main conceptual procedures.

## 1.2. Running Code Cells
Select any code cell and click on the (▶) sign in the top-left of the cell's frame to run its code (note that code cells have to be run in order when running for the first time).

In [1]:
# First install the `pept` library using pip, Python's package manager
!pip install pept

# 2. PEPT-EM

## 2.1. Read in Line of Response Data
In this initial step, we read in the "raw" LoR data from which our tracer positions are calculated. In this simple example, we load only a single sample (i.e. one particle location at one point in time). The LoRs corresponding to this sample are output as an image.

As discussed in the main text, varying the numbers of LoRs used to calculate a PEPT location can affect the quality of the final location measured. Try altering the value "nrows" below to see how this inluences your results.

In [2]:
# Read in a sample of experimental PEPT data from an online repository into a NumPy array
import numpy as np
import pept

# Skip the file header's first 15 lines, then read in 50 LoRs
lors_raw = pept.utilities.read_csv(
    "https://raw.githubusercontent.com/uob-positron-imaging-centre/example_data/master/sample_1p_fluidised_bed.csv",
    skiprows = 15,
    nrows = 50,
)

# Insert columns for the z-coordinates
head_separation = 600

lors_raw = np.insert(lors_raw, 3, 0, axis = 1)
lors_raw = np.insert(lors_raw, 6, head_separation, axis = 1)

# Project the 3D lines onto the YZ plane for ease of analysis - i.e. select only columns
# [time, y1, z1, y2, z2] and flip columns to get [time, x1, y1, x2, y2]
lors = np.array(lors_raw[:, [0, 3, 2, 6, 5]])

# Print the line of response (LoR) data
lors

array([[0.000e+00, 0.000e+00, 1.687e+02, 6.000e+02, 1.428e+02],
       [1.000e-01, 0.000e+00, 1.676e+02, 6.000e+02, 3.139e+02],
       [1.000e-01, 0.000e+00, 4.100e+02, 6.000e+02, 2.401e+02],
       [2.000e-01, 0.000e+00, 2.962e+02, 6.000e+02, 4.525e+02],
       [2.000e-01, 0.000e+00, 1.151e+02, 6.000e+02, 3.534e+02],
       [2.000e-01, 0.000e+00, 1.322e+02, 6.000e+02, 2.661e+02],
       [2.000e-01, 0.000e+00, 3.735e+02, 6.000e+02, 1.310e+02],
       [3.000e-01, 0.000e+00, 1.115e+02, 6.000e+02, 3.534e+02],
       [4.000e-01, 0.000e+00, 2.094e+02, 6.000e+02, 2.808e+02],
       [4.000e-01, 0.000e+00, 2.749e+02, 6.000e+02, 3.062e+02],
       [5.000e-01, 0.000e+00, 2.389e+02, 6.000e+02, 1.681e+02],
       [6.000e-01, 0.000e+00, 4.283e+02, 6.000e+02, 7.020e+01],
       [6.000e-01, 0.000e+00, 1.333e+02, 6.000e+02, 3.499e+02],
       [6.000e-01, 0.000e+00, 3.050e+02, 6.000e+02, 1.032e+02],
       [6.000e-01, 0.000e+00, 9.200e+01, 6.000e+02, 3.929e+02],
       [6.000e-01, 0.000e+00, 1.859e+02,

In [3]:
from pept.visualisation import PlotlyGrapher2D

grapher = PlotlyGrapher2D()
grapher.add_lines(lors)
grapher.show()

## 2.2 Represent the LoRs as Vectors

In [4]:
# Rewrite LoRs in the vectorial form y(x) = position + x * direction
lors_original = np.copy(lors)

lors[:,3] = lors[:,3] - lors[:,1]
lors[:,4] = lors[:,4] - lors[:,2]

lors[:,3:] /= np.linalg.norm(lors[:,3:], axis=-1)[:,np.newaxis]

## 2.3. Calculate MDP and Assign a Weight to each LoR

The next step of the process is to calculate the minimum distance point (MDP) for the set of LoRs currently under consideration, much as we did for the Birmingham algorithm (see main text).

Then, using this information, each LoR is assigned a weight according to its distance from the MDP, with lines lying closer to this point being assigned a greater weight, and those further away a lesser weight. 

In the plot below, the weight of a line is represented by its opacity, with more heavily-weighted LoRs being more opaque, and less-heavily-weighted LoRs being more transparent. The size of the red circle representing the minimum distance point corresponds to its variance, a measure of how strongly different LoRs' distances from the MDP vary - if all LoRs lie close to the MDP (as one would expect if only true LoRs were present) one would expect a small variance.

In [5]:
# Calculate an initial guess for the location (essentially a minimum distance point) for these LoRs
from numpy import newaxis as nx

def centroid(lors, weights):
    m = (np.identity(2)[nx,:,:]-lors[:,nx,3:]*lors[:,3:,nx])*weights[:,nx,nx]
    M = np.sum(m,axis=0)
    V = np.sum(np.sum(m*lors[:,np.newaxis,1:3],axis=-1),axis=0)
    x = np.matmul(np.linalg.inv(M),V)
    return x

def dist_matrix(x,lors):
    X = x[np.newaxis,:3]-lors[:,1:3]    
    d2 = np.sum(X**2,axis=-1)-np.sum(X*lors[:,3:],axis=-1)**2
    return d2

def latent_weights(d2,s,eps=0,r=None):
    if r is None:
        w = np.exp(-d2/2/s**2)/s**2+10**(-20)
        w /= np.sum(w)+eps
    else:
        w = np.exp(-d2/2/s**2)*r/s**2+10**(-20)
        w/= np.sum(w)+eps*(1-np.sum(r))
    return w

# Begin with equal weights for all LoRs
weights = np.ones(len(lors))

# Calculate MDP and distance from it to all LoRs
x = centroid(lors, weights)
d2 = dist_matrix(x, lors)
variance = np.sqrt(np.mean(d2))

# Then recalculate the weight of each LoR for the given centroid
weights = latent_weights(d2, variance, eps=0.0001)

# Set the centroid's timestamp as the mean timestamp of the LoRs used
centre = np.insert(x, 0, lors[:, 0].mean())

print(f"Before iterating: Centroid = {x} mm, Variance = {variance} mm")

Before iterating: Centroid = [296.54123338 243.38995732] mm, Variance = 45.41596057401464 mm


In [6]:
# Plot LoRs, setting each line's opacity as its relative weight. The centroid's size is given by
# its variance
grapher = PlotlyGrapher2D()

for i in range(len(lors_original)):
    grapher.add_lines(lors_original[i:i+1,:], opacity=0.6*weights[i]/weights.max(), color='blue')

grapher.add_points([centre], size=variance)

grapher.show()

## 2.4 Recalculate MDP with Previous Weights

The above step is then repeated, iteratively recalculating the centroid and variance from the weighted LoRs, which themselves are also recalculated at each step. As can be seen both from the variance printed at the end of the below section and the relative size of the red dot compared to the first iteration, this process markedly reduces the impact of 'outlier' LoRs.

Once finished iterating, the particle position can be taken simply as the finally-computed MDP.

In [7]:
x = centroid(lors, weights)
d2 = dist_matrix(x, lors)
variance = np.sqrt(np.mean(d2*weights))
weights = latent_weights(d2, variance, eps=0.0001)

print(f"After iterating: Centroid = {x} mm, Variance = {variance} mm")

# Set the centroid's timestamp as the mean timestamp of the LoRs used
centre = np.insert(x, 0, lors[:, 0].mean())

After iterating: Centroid = [310.83598108 244.13049833] mm, Variance = 3.164902298173844 mm


In [8]:
# Plot LoRs, setting each line's opacity as its relative weight. The centroid's size is given by
# its variance
grapher = PlotlyGrapher2D()

for i in range(len(lors_original)):
    grapher.add_lines(lors_original[i:i+1,:], opacity=0.6*weights[i]/weights.max(), color='blue')

grapher.add_points([centre], size=variance)

grapher.show()

In this instance, we see that the point converges very quickly and the PEPT-EM algorithm reduces the weight of outlier LORs. In the case of multiple particles, we may need more iterations in order to converge on the correct points.

# 3. Complete PEPT-EM Code

In [9]:
# Read in a sample of experimental PEPT data from an online repository into a NumPy array
import numpy as np
import pept

# Skip the file header's first 15 lines, then read in 50 LoRs
lors_raw = pept.utilities.read_csv(
    "https://raw.githubusercontent.com/uob-positron-imaging-centre/example_data/master/sample_1p_fluidised_bed.csv",
    skiprows = 15,
    nrows = 1000,
)

# Insert columns for the z-coordinates
head_separation = 600

lors_raw = np.insert(lors_raw, 3, 0, axis = 1)
lors_raw = np.insert(lors_raw, 6, head_separation, axis = 1)

# Project the 3D lines onto the YZ plane for ease of analysis - i.e. select only columns
# [time, y1, z1, y2, z2] and flip columns to get [time, x1, y1, x2, y2]
lors = np.array(lors_raw[:, [0, 3, 2, 6, 5]])

# Print the line of response (LoR) data
lors

array([[0.000e+00, 0.000e+00, 1.687e+02, 6.000e+02, 1.428e+02],
       [1.000e-01, 0.000e+00, 1.676e+02, 6.000e+02, 3.139e+02],
       [1.000e-01, 0.000e+00, 4.100e+02, 6.000e+02, 2.401e+02],
       ...,
       [3.560e+01, 0.000e+00, 8.440e+01, 6.000e+02, 2.384e+02],
       [3.570e+01, 0.000e+00, 5.463e+02, 6.000e+02, 7.670e+01],
       [3.570e+01, 0.000e+00, 5.334e+02, 6.000e+02, 7.020e+01]])

In [10]:
from numpy import newaxis as nx

# User-defined FPI settings
sample_size = 100           # Number of LoRs in a sample
target_variance = 0.1       # Target variance of tracer location found, in mm


def centroid(lors, weights):
    m = (np.identity(2)[nx,:,:]-lors[:,nx,3:]*lors[:,3:,nx])*weights[:,nx,nx]
    M = np.sum(m,axis=0)
    V = np.sum(np.sum(m*lors[:,np.newaxis,1:3],axis=-1),axis=0)
    x = np.matmul(np.linalg.inv(M),V)
    return x

def dist_matrix(x,lors):
    X = x[np.newaxis,:3]-lors[:,1:3]    
    d2 = np.sum(X**2,axis=-1)-np.sum(X*lors[:,3:],axis=-1)**2
    return d2

def latent_weights(d2,s,eps=0,r=None):
    if r is None:
        w = np.exp(-d2/2/s**2)/s**2+10**(-20)
        w /= np.sum(w)+eps
    else:
        w = np.exp(-d2/2/s**2)*r/s**2+10**(-20)
        w/= np.sum(w)+eps*(1-np.sum(r))
    return w


sample_start = 0
positions = []

# Rewrite LoRs in the vectorial form y(x) = position + x * direction
lors_original = np.copy(lors)

lors[:,3] = lors[:,3] - lors[:,1]
lors[:,4] = lors[:,4] - lors[:,2]

lors[:,3:] /= np.linalg.norm(lors[:,3:], axis=-1)[:,np.newaxis]

while sample_start + sample_size < len(lors):
    sample = lors[sample_start:sample_start + sample_size]

    # Begin with equal weights for all LoRs in sample
    weights = np.ones(len(sample))

    # Calculate MDP and distance from it to all LoRs in sample
    x = centroid(sample, weights)
    d2 = dist_matrix(x, sample)
    variance = np.sqrt(np.mean(d2))

    # Then recalculate the weight of each LoR for the given centroid
    weights = latent_weights(d2, variance, eps=0.0001)

    # Iteratively recalculate MDP, LoR weights and variance of tracer location
    while variance < target_variance:
        x = centroid(sample, weights)
        d2 = dist_matrix(x, sample)
        variance = np.sqrt(np.mean(d2*weights))
        weights = latent_weights(d2, variance, eps=0.0001)

    # Set the centroid's timestamp as the mean timestamp of the LoRs used
    centre = np.insert(x, 0, sample[:, 0].mean())
    positions.append(centre)

    sample_start += sample_size

positions = np.array(positions)
positions

array([[  1.931     , 297.30957115, 253.2870968 ],
       [  5.878     , 339.25565921, 252.07824896],
       [  9.966     , 311.80423304, 268.91514561],
       [ 13.588     , 295.15875411, 254.877997  ],
       [ 17.084     , 306.09961655, 251.81577586],
       [ 20.843     , 269.46710909, 259.39081263],
       [ 24.6       , 292.08878172, 256.43684004],
       [ 28.244     , 298.63708675, 261.53063642],
       [ 31.356     , 280.90691406, 260.39716994]])

In [11]:
grapher = PlotlyGrapher2D(cols = 2)

grapher.add_lines(lors_original[:400])
grapher.add_points(positions, col = 2)

grapher.show()